In [1]:
import pandas as pd
import numpy as np
import copy, time
import math
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, KFold ,train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

df_train = pd.read_csv('./train_data.csv')
df_test = pd.read_csv('./test_features.csv')
print(df_train.shape)
columns=list(df_train.columns)
print(columns)
df_train.head()

(113, 22)
['name', 'bonus', 'deferral_payments', 'deferred_income', 'director_fees', 'email_address', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances', 'long_term_incentive', 'other', 'poi', 'restricted_stock', 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']


,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [2]:
train_Y=df_train['poi'] #目標欄位
name = df_test['name'] 
df_train = df_train.drop(['name','poi'] , axis=1) 
columns.remove('name')
columns.remove('poi')
df_test = df_test.drop(['name'] , axis=1)
df = pd.concat([df_train, df_test],sort=False) #資料一併做前處理
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


 'director_fees', 'exercised_stock_options', 'expenses', 'long_term_incentive','restricted_stock', 'restricted_stock_deferred',  'total_payments', 'total_stock_value' 

In [3]:
#df['BAXTER JOHN C'].loc[] = 300000
#df['LAY KENNETH L'].loc = 

#### 缺失值處理

In [4]:
def na_num(df_data):
    num = (df_data.isnull().sum()/len(df)).sort_values(ascending=False)
    return num

print(na_num(df)) #欄位缺失率

#缺失值比率過高，先捨去欄位
df = df.drop(['loan_advances'],axis=1)

#類別缺失值處理，改以是否有信箱
none_cols = ['email_address']
for col in none_cols:
    df[col]=df[col].isnull() 
    #df[col] = df[col].fillna("None")

#部分欄位缺失值填補0
zero_cols = ['long_term_incentive', 'deferred_income','deferral_payments','director_fees',\
            'from_messages','to_messages','from_poi_to_this_person', 'from_this_person_to_poi',\
            'shared_receipt_with_poi',\
             'director_fees', 'exercised_stock_options', 'expenses', 'long_term_incentive','restricted_stock',\
             'restricted_stock_deferred', 'total_payments', 'total_stock_value' ]
for col in zero_cols:
    df[col] = df[col].fillna(0)

#部分欄位缺失值填補眾數
mod_cols = []
for col in mod_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

#部分欄位缺失值填補平均數
mean_cols = ['other','exercised_stock_options']
for col in mean_cols:
    df[col] = df[col].fillna(df[col].mean())

#部分欄位缺失值填補中位數
med_cols = ['salary','bonus']
for col in med_cols:
    df[col] = df[col].fillna(df[col].median())

#確認缺失值填補是否完整
print(na_num(df)) 

loan_advances                0.972603
director_fees                0.883562
restricted_stock_deferred    0.876712
deferral_payments            0.732877
deferred_income              0.664384
long_term_incentive          0.547945
bonus                        0.438356
from_messages                0.410959
from_poi_to_this_person      0.410959
from_this_person_to_poi      0.410959
shared_receipt_with_poi      0.410959
to_messages                  0.410959
other                        0.363014
expenses                     0.349315
salary                       0.349315
exercised_stock_options      0.301370
restricted_stock             0.246575
email_address                0.239726
total_payments               0.143836
total_stock_value            0.136986
dtype: float64
total_stock_value            0.0
from_poi_to_this_person      0.0
deferral_payments            0.0
deferred_income              0.0
director_fees                0.0
email_address                0.0
exercised_stock_options    

#### 離群值處理

In [5]:
#離群值處理
def remove_outlier(df):
    for k in df.columns[1:]:
        df_temp = df[k].copy()
        df_temp_iqr = iqr(df_temp)
        df_temp_std = np.std(df_temp)
        df_temp_med= np.median(df_temp)
        df_temp_q1=np.percentile(df_temp , 25)
        df_temp_q3=np.percentile(df_temp , 75)
        df_temp_lower = df_temp_q1-1.5*df_temp_iqr
        df_temp_upper = df_temp_q3+2*df_temp_iqr
        #df[k].fillna(df[k].median(),inplace=True)    
        df[k].loc[df_temp > df_temp_upper]=np.percentile(df_temp, q = 99.5)

#### Scale

In [6]:
# 將資料最大最小化
#df = MinMaxScaler().fit_transform(df)
#將資料標準化
df = StandardScaler().fit_transform(df)

# 將前述轉換完畢資料 df , 重新切成 train_X, test_X
train_num = train_Y.shape[0]
train_X = pd.DataFrame(df[:train_num])
test_X = pd.DataFrame(df[train_num:])

train_Y = pd.DataFrame(train_Y.map(lambda x :int(x)))

#### 切分數據集

In [7]:
kf=KFold(n_splits = 5, shuffle = True, random_state = 42)
result=next(kf.split(train_X), None)
train=train_X.iloc[result[0]]
valid=train_X.iloc[result[1]]
df_train=train
df_train_Y=train_Y.iloc[result[0]]
df_valid=valid
df_valid_Y=train_Y.iloc[result[1]]

df_train, df_valid, df_train_Y,df_valid_Y = train_test_split(train_X, train_Y,\
test_size=0.2, random_state=42)

In [8]:
# 使用模型 : 邏輯斯迴歸 / Lasso / Ridge / 梯度提升 / 隨機森林
from sklearn.linear_model import LogisticRegression,Ridge,Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
#from sklearn.model_selection import GridSearchCV
t = lambda x :1 if x >1 else x
def tra(y):
    for i in range(len(y)):
        y[i]=t(y[i])
    return y

In [9]:
import xgboost as xgb
param = {'max_depth':5, 'eta':1, 'objective':'binary:logistic', 'alpha': 0.0001, 'lambda': 1}
num_round = 15
dtrain = xgb.DMatrix(df_train,label=df_train_Y)
xgbst = xgb.train(param, dtrain, num_round)

dval= xgb.DMatrix(df_valid,label=df_valid_Y)

xgbst_predval = xgbst.predict(dval)
validlabel = dval.get_label()
AUC = metrics.roc_auc_score(validlabel,xgbst_predval)
print(AUC)
# make prediction
dtrain = xgb.DMatrix(train_X,label=train_Y)
xgbst = xgb.train(param, dtrain, num_round)
dtest = xgb.DMatrix(test_X)
xgbst_preds = xgbst.predict(dtest)
print(xgbst_preds)
sub = pd.DataFrame({'name': name, 'poi': xgbst_preds})
print(sub)
#sub.to_csv('Enron_xgboost.csv', index=False) 

0.8444444444444444
[1.2788171e-01 4.4757742e-01 9.7544122e-01 3.6823396e-02 6.1185557e-01
 4.3154737e-01 8.4746975e-01 1.0397176e-03 1.2610568e-01 5.7222150e-02
 1.7416434e-02 4.7388097e-04 1.2405521e-03 1.0397176e-03 7.4563855e-01
 1.0397176e-03 2.0573473e-01 2.6708648e-01 2.0335475e-01 4.7388097e-04
 7.5608534e-01 7.4172175e-01 1.4301986e-02 4.6083583e-03 1.2046383e-02
 1.2092295e-02 2.1023048e-02 3.3932840e-03 3.9750543e-01 6.9253795e-02
 2.5566248e-03 5.4356390e-01 5.6015611e-01]
                    name       poi
0       BELDEN TIMOTHY N  0.127882
1     BOWEN JR RAYMOND M  0.447577
2         HANNON KEVIN P  0.975441
3       DELAINEY DAVID W  0.036823
4       CAUSEY RICHARD A  0.611856
5       HICKERSON GARY J  0.431547
6         FREVERT MARK A  0.847470
7            CHAN RONNIE  0.001040
8   DONAHUE JR JEFFREY M  0.126106
9      REYNOLDS LAWRENCE  0.057222
10      HORTON STANLEY C  0.017416
11         LEWIS RICHARD  0.000474
12    SHERRICK JEFFREY B  0.001241
13        MEYER JEROM

In [10]:
ridge = Ridge(alpha=.3)

###########train###########
ridge.fit(df_train,df_train_Y)
###########valid###########
ridge_predval =ridge.predict(df_valid).reshape(23)
print(ridge_predval)
AUC = metrics.roc_auc_score(df_valid_Y, ridge_predval)
print(AUC)

###########tset###########
#ridge.fit(train_X, train_Y)
ridge_pred = abs(ridge.predict(test_X)).reshape(33)

sub = pd.DataFrame({'name': name, 'poi': ridge_pred})
print(sub)
#sub.to_csv('Enron_ridge.csv', index=False) 

[ 0.14659487  0.17167216 -0.00496693  0.2288889   0.14840997  0.23441053
 -0.01299199  0.07561749  0.22348156  0.07275    -0.02858571  0.29619235
  0.28467316  0.0186774   0.01729695  0.54199224 -0.02110415  0.02665715
  0.09547894  0.0523067   0.04329144  0.10669793  0.10645565]
0.7111111111111111
                    name        poi
0       BELDEN TIMOTHY N   0.059101
1     BOWEN JR RAYMOND M   0.171396
2         HANNON KEVIN P   0.150570
3       DELAINEY DAVID W   0.293580
4       CAUSEY RICHARD A   0.091729
5       HICKERSON GARY J   0.153842
6         FREVERT MARK A   0.219129
7            CHAN RONNIE   0.028725
8   DONAHUE JR JEFFREY M   0.317963
9      REYNOLDS LAWRENCE   0.058727
10      HORTON STANLEY C   0.049238
11         LEWIS RICHARD   0.037088
12    SHERRICK JEFFREY B   0.070255
13        MEYER JEROME J   0.015910
14   DERRICK JR. JAMES V   0.145619
15              PIRO JIM   0.037381
16   DETMERING TIMOTHY J   0.083328
17     KISHKILL JOSEPH G   0.148027
18        BAY FR

In [11]:
lasso = Lasso(alpha=0.01)
###########train###########
lasso.fit(df_train,df_train_Y)
###########valid###########
lasso_predval = tra(abs(lasso.predict(df_valid)))
AUC=metrics.roc_auc_score(df_valid_Y, lasso_predval)
print(AUC)
###########tset###########
#lasso = Lasso(alpha=0.001)
#lasso.fit(train_X, train_Y)
lasso_pred = abs(lasso.predict(test_X))
sub = pd.DataFrame({'name': name, 'poi': tra(lasso_pred)})
print(sub)
#sub.to_csv('Enron_lasso.csv', index=False) 

0.6555555555555556
                    name       poi
0       BELDEN TIMOTHY N  0.126890
1     BOWEN JR RAYMOND M  0.112855
2         HANNON KEVIN P  0.161544
3       DELAINEY DAVID W  0.128235
4       CAUSEY RICHARD A  0.080971
5       HICKERSON GARY J  0.078662
6         FREVERT MARK A  0.349543
7            CHAN RONNIE  0.007183
8   DONAHUE JR JEFFREY M  0.145352
9      REYNOLDS LAWRENCE  0.079555
10      HORTON STANLEY C  0.164121
11         LEWIS RICHARD  0.080243
12    SHERRICK JEFFREY B  0.094147
13        MEYER JEROME J  0.007184
14   DERRICK JR. JAMES V  0.230768
15              PIRO JIM  0.061149
16   DETMERING TIMOTHY J  0.097685
17     KISHKILL JOSEPH G  0.062162
18        BAY FRANKLIN R  0.062261
19        PRENTICE JAMES  0.077395
20                 TOTAL  1.000000
21        FALLON JAMES B  0.081862
22            METTS MARK  0.077039
23         WODRASKA JOHN  0.061743
24   MORDAUNT KRISTINA M  0.062100
25      CARTER REBECCA C  0.070502
26      BHATNAGAR SANJAY  0.118783
2

In [12]:
# 線性迴歸預測檔
lg = LinearRegression()
###########train###########
lg.fit(df_train,df_train_Y)
#print(lg.coef_)
###########valid###########
lg_predval = tra(abs(lg.predict(df_valid)))
AUC=metrics.roc_auc_score(df_valid_Y, lg_predval)
print(AUC)

###########tset###########
#lg.fit(train_X, train_Y)
lg_pred = abs(lg.predict(test_X))
sub = pd.DataFrame({'name': name, 'poi': tra(lg_pred)})
print(sub)
#sub.to_csv('Enron_linear.csv', index=False) 

0.7222222222222222


Exception: Data must be 1-dimensional

In [ ]:
# 梯度提升機預測檔 
gdbt = GradientBoostingRegressor(tol=0.1, subsample=0.37, n_estimators=500, #max_features=20, 
                                 max_depth=8, learning_rate=0.01)
###########train###########
gdbt.fit(df_train,df_train_Y)
###########valid###########
gdbt_predval = tra(abs(gdbt.predict(df_valid)))
AUC=metrics.roc_auc_score(df_valid_Y, gdbt_predval)
print(AUC)

###########tset###########
gdbt.fit(train_X, train_Y)
gdbt_pred = abs(gdbt.predict(test_X))

sub = pd.DataFrame({'name': name, 'poi': tra(gdbt_pred)})
#print(sub)
#sub.to_csv('Enron_gdbt.csv', index=False)

In [ ]:
# 隨機森林預測檔 
rf = RandomForestRegressor(n_estimators=200, min_samples_split=8, min_samples_leaf=10,#max_features='', 
                           max_depth=6, bootstrap=False)
###########train###########
rf.fit(df_train,df_train_Y)
###########validation###########
rf_predval = abs(rf.predict(df_valid))
AUC=metrics.roc_auc_score(df_valid_Y, rf_predval)
print(AUC)
###########test###########
rf.fit(train_X, train_Y)
rf_pred = abs(rf.predict(test_X))
sub = pd.DataFrame({'name': name, 'poi': tra((rf_pred))})
#print(sub)
#sub.to_csv('Enron_rf.csv', index=False)

In [ ]:
log_r=LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0)
log_r.fit(df_train,df_train_Y)
log_r_predval = abs(log_r.predict_proba(df_valid))
log_r_pred=[]
for i,p in enumerate(log_r_predval):
    log_r_pred.append(log_r_predval[i][1])
log_r_pred=np.array(log_r_pred)
print(log_r_pred)
AUC=metrics.roc_auc_score(df_valid_Y, log_r_pred)
print(AUC)
###########test###########
log_r=LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0)
log_r.fit(train_X, train_Y)
log_r_pred=log_r.predict_proba(test_X)
print(log_r_pred)
pred=[]
for i,p in enumerate(log_r_pred):
    pred.append(log_r_pred[i][1])
pred=np.array(pred)
sub = pd.DataFrame({'name': name, 'poi':pred})
print(sub)
#sub.to_csv('Enron_logistic.csv', index=False)

In [ ]:
# 混合泛化預測
blending_pred = ridge_pred*0.05+ rf_pred*0.75+xgbst_preds*0.2
sub = pd.DataFrame({'name': name, 'poi':tra(blending_pred)})
#sub['poi'] = sub['poi'].map(lambda x:True if x>0.5 else False) 
print(sub)
#sub.to_csv('rf-xgbst-ridge.csv', index=False)